In [ ]:
import os
import requests
import os.path
import csv
import json
import pytz
import pygsheets

from datetime import datetime, timedelta
import pandas as pd


headers = {
    'X-PW-AccessToken':os.environ["COPPER_API_KEY"],
    'X-PW-Application':'developer_api',
    'X-PW-UserEmail':'cooper@rehabpath.com',
    'Content-Type':'application/json'
}

def refresh_custom_fields(return_results = False):
    """
    Function to refresh the custom_field_json file and related dictionary. Option to 
    return resulting dictionary:

    return_results: True to return results, False to just update local file
    """
    # Fetching custom fields:
    list_fields = requests.get('https://api.copper.com/developer_api/v1/custom_field_definitions',headers=headers)
    
    # Was the API request a success?
    if list_fields.status_code != 200:
        print('Error fetching custom fields.')
        if return_results:
            print('Returning previous version.')
            with open('custom_fields.json', encoding="utf-8") as f:
                return json.load(f)
    
    # Obtaining response text and initializing dictionary:
    list_fields_json = list_fields.json()
    custom_field_dict = {}

    for item in list_fields_json:
        item_dict = {}
        for sub_item in ['name','data_type','available_on','is_filterable']:
            item_dict[sub_item] = item[sub_item] 

        item_options = {}

        if 'options' in item:
            for sub_item in item['options']:
                sub_item_name = sub_item['name']
                sub_item_id = sub_item['id']

                item_options[sub_item_id] = sub_item_name
        
            item_dict['options'] = item_options
        item_id = item['id']
        custom_field_dict[item_id] = item_dict
    
    # Updating local file:
    with open('custom_fields.json', 'w') as file:
        json.dump(custom_field_dict, file, indent=4)
    
    #Option to return results or just update local file:
    if return_results == True:
        return custom_field_dict
    else:
        print('Successfully Updated File!')
        pass

custom_fields = refresh_custom_fields(True)

def get_cf_info(cf_id,cf_info):
    """
    Function to get the custome field information based on the field id.

    cf_id (int or string) is the id_number of the custome field
    cf_info can be a single string or list of strings like 'name' or ['name','data_type','is_filterable']
    """

    if isinstance(cf_id,str):
        cf_id = int(cf_id)      

    # Check if cf_info item(s) are valid
    if isinstance(cf_info, list):
        for item_name in cf_info:
            # Remove faulty items:
            if item_name not in ['name','data_type','available_on','is_filterable','options']:
                cf_info.pop(item_name)
                print(f'Invalid cf_info: {cf_info}')
            
            # Cancel function if no valid items
            if len(cf_info) == 0:
                pass
    elif cf_info not in ['name','data_type','available_on','is_filterable','options']:
        print(f'Invalid cf_info: {cf_info}'); pass

    if isinstance(cf_info,list):
        return_list = []
        for item_name in cf_info:
            value = custom_fields[cf_id].get(item_name)
            return_list.append(value)
        return return_list
    else:
        value = custom_fields[cf_id].get(cf_info)
        return value

def get_cf_options(cf_id):
    return custom_fields.get(cf_id).get('options')

def cf_option_name(cf_id,option_id):
    return get_cf_options(cf_id).get(option_id)

def reformat_company_data(company_data):
    """
    Function to reformat the company data recieved from Copper in their JSON response
    """

    native_items = ['id', 'name', 'address', 'assignee_id', 'contact_type_id']

    output_dict = {}

    for item in native_items:
        output_dict[item] = company_data.get(item, None)
    
    custom_field_data = company_data['custom_fields']

    for dict_item in custom_field_data:
        item_id = dict_item['custom_field_definition_id']
        item_name = custom_fields_dict.get(item_id, None)

        if item_name not in Custom_Fields or item_name is None:
            continue
        
        if item_name is not None:
            item_value = dict_item['value']
            output_dict[item_name] = item_value
    
    return output_dict

def write_to_gsheet(service_file_path, spreadsheet_id, sheet_name, data_df):
    """
    this function takes data_df and writes it under spreadsheet_id
    and sheet_name using your credentials under service_file_path
    """
    gc = pygsheets.authorize(service_file=service_file_path)
    sh = gc.open_by_key(spreadsheet_id)
    try:
        sh.add_worksheet(sheet_name)
    except:
        pass
    wks_write = sh.worksheet_by_title(sheet_name)
    wks_write.clear('A1',None,'*')
    wks_write.set_dataframe(data_df, (1,1), encoding='utf-8', fit=True)
    wks_write.frozen_rows = 1

custom_fields_dict = {}

for item in custom_fields.keys():
    item_name = custom_fields[item].get('name')
    custom_fields_dict[item] = item_name

custom_fields_df = pd.DataFrame(list(custom_fields_dict.items()),columns = ['id','name'])

custom_fields_list = list(custom_fields_df['name'])

# Daily Job to Get PPV Values

Daily job to pull GA4 pageviews from snowflake and combine it with data from Copper to get the total PPVs by day for all advertier and send the results to `AdvertisingPartners_RatesAndBudgets`.

**Data from Snowflake:**
- Pageviews by `slug` by day

**Data from Coppper:**
- Center `slug` - to pair to GA4 data
- Center `GoLive Date` - to discard PPVs before a advertier goes GoLive
- Center `Page` - similar to slug, but how the data was been historically reported.

### Getting Data from Copper

Let's start by getting the Copper data for all companies with the status of `Advertiser`, `Onboarding-CS`,`Onboarding-AdOps`, `Paused`, or `Churning`.  We can do this by searching for comapnies with these values in the field. Note, that we need to use the id for the custome field, 588393, and the ids for all of the statuses.

In [ ]:
total_pages = page = 1
combined_results = []

while page <= total_pages:
    page_params = {
        "page_size": 100,
        "page_number": page,
        "custom_fields": [
            {"custom_field_definition_id": 588393,                  # Company Status - custom field
            "value": [1806073, 1806063,1806064,1824709,1806067]    # List of ids for allowed statuses
        }
        ]}
    result = requests.post('https://api.copper.com/developer_api/v1/companies/search',headers=headers,json=page_params)
    
    if result.status_code == 200:
        total_pages = (int(result.headers['X-PW-TOTAL'])//100)+1
        print(f"Getting page {page} of {total_pages}.")
        result_json = result.json()
        combined_results.extend(result_json)
        page +=1

    else:
        print(f"Issue with page {page}. Stopping.")
        break

Getting page 1 of 3.
Getting page 2 of 3.
Getting page 3 of 3.


Now that we have the data we want, we need to do some data cleaning to dates and custom fields and discard information we don't want. First, let's loop through the results, which are in a json/dict format, turn them into a flat list, and finally turn the list into a dataframe.

In [ ]:
Custom_Fields = ['Page','GoLive Date','Profile Slug']

In [ ]:
cleaned_data = []

for idx in range(len(combined_results)):
    results = reformat_company_data(combined_results[idx]) # using a function I wrote. See hidden section.
    cleaned_data.append(results)

cleaned_data_df = pd.DataFrame(cleaned_data)
list_slugs = list(set(cleaned_data_df['Profile Slug']))
cleaned_data_df.drop(columns=['id','name','address','assignee_id','contact_type_id'],inplace=True)

Now that we have a dataframe, let's change the data formate from a unix timestamp to a human readable date.

In [ ]:
ct_timezone = pytz.timezone('US/Central')
cleaned_data_df['GoLive Date'] = cleaned_data_df['GoLive Date'].apply(lambda x: pd.to_datetime(x, unit='s'))
cleaned_data_df

,Profile Slug,GoLive Date,Page
0,1000-islands,1999-12-28 06:00:00,/1000-islands/
1,90210-recovery-california,1999-12-31 06:00:00,/90210-recovery-california/
2,abhasa-rehabilitation-thondamuthur-india,2023-03-25 05:00:00,/abhasa-rehabilitation-thondamuthur-india/
3,None,2023-02-28 06:00:00,None
4,abhasa-wellness-retreat,2022-10-16 05:00:00,/abhasa-wellness-retreat/
...,...,...,...
293,wish-recovery-california,2023-07-14 05:00:00,/wish-recovery-california/
294,within-center-austin-texas,2023-10-08 05:00:00,/within-center-austin-texas/
295,zeus-rehab-warsaw-poland,2024-01-04 23:00:00,/zeus-rehab-warsaw-poland/
296,zinnia-healing-singer-island,2023-05-05 05:00:00,/zinnia-healing-singer-island/


### Getting Views from Snowflake

We can pull all of the pageviews / ppvs data from snowflake by inserting dynamic values into the SQL query. We want all of the data for yesterday's month and year. This get's the whole month of data even when yesterday was the last day of the month.

In [ ]:
yesterday = datetime.now(ct_timezone)-timedelta(1)
search_year = yesterday.year
search_month = yesterday.month

We can insert dynamic values into the SQL querry by using double curly bracekts {{example}}.

In [ ]:
views_df = _deepnote_execute_sql('SELECT * \nFROM RP_PROD_DL."CLEAN_TABLES_GA4".CENTER_DAILY_VIEWS \nWHERE YEAR(AS_OF_DATE) = {{search_year}} AND MONTH(AS_OF_DATE) = {{search_month}};', 'SQL_5E90C4C2_A521_4993_A1E9_D77837A4FD79', audit_sql_comment='', sql_cache_mode='cache_disabled')
views_df

,as_of_date,slug,property_id,region,city,number_of_views
0,2024-01-04,browse,278764105,Florida,Orlando,14
1,2024-01-04,houston,278764105,Texas,Houston,8
2,2024-01-04,,278764105,California,Irvine,3
3,2024-01-04,ohlhoff-california,278764105,California,San Francisco,3
4,2024-01-04,condition,278764105,Texas,Keller,2
...,...,...,...,...,...,...
38074,2024-01-03,therapies,278764105,California,Camarillo,3
38075,2024-01-03,,278764105,Massachusetts,Monterey,2
38076,2024-01-02,wisconsin,278764105,Wisconsin,Waukesha,1
38077,2024-01-02,searidge-foundation-canada,278764105,Nova Scotia,Dartmouth,1


Now that we have the data, let's again do some data cleaning. We need to turn `as_of_date` into a date datatype, discard data for non-advertisers and listing pages, and transform the data into a table.

In [ ]:
# Formating Date:
views_df['as_of_date'] = views_df['as_of_date'].astype(str)

# Discarding unneeded data:
filtered_views = views_df[views_df['slug'].isin(list_slugs)][['as_of_date', 'slug', 'number_of_views']]

# Formating into a table:
pivoted_views = filtered_views.groupby(['slug', 'as_of_date'])['number_of_views'].sum().unstack().reset_index()
pivoted_views.rename(columns={'slug':'Profile Slug'},inplace=True)
pivoted_views

as_of_date,Profile Slug,2024-01-01,2024-01-02,2024-01-03,2024-01-04,2024-01-05,2024-01-06,2024-01-07,2024-01-08
0,1000-islands,28.0,18.0,18.0,12.0,10.0,13.0,27.0,6.0
1,90210-recovery-california,5.0,2.0,4.0,4.0,2.0,6.0,7.0,NaN
2,abhasa-rehabilitation-india,15.0,11.0,7.0,8.0,7.0,12.0,16.0,1.0
3,abhasa-rehabilitation-thondamuthur-india,8.0,6.0,3.0,6.0,1.0,12.0,11.0,NaN
4,abhasa-wellness-retreat,7.0,5.0,7.0,2.0,3.0,13.0,12.0,1.0
...,...,...,...,...,...,...,...,...,...
250,wilmington-treatment-center-north-carolina,1.0,3.0,NaN,2.0,2.0,3.0,1.0,NaN
251,wish-recovery-california,21.0,13.0,16.0,17.0,17.0,33.0,30.0,NaN
252,within-center-austin-texas,4.0,3.0,11.0,13.0,8.0,8.0,19.0,1.0
253,zeus-rehab-warsaw-poland,5.0,1.0,2.0,3.0,7.0,13.0,12.0,NaN


### Removing PPVs Before GoLive Dates

Our PPV data is now ready to be combined with the GoLive dates. This will allow us to search whether a column is before a center's GoLive date. If it is, we're going to set the PPVs on that day to zero.

In [ ]:
# Mergeing/joining our two dataframes:
merged_df = pd.merge(pivoted_views, cleaned_data_df, on='Profile Slug', how='left')
list_cols = merged_df.columns[1:-1]

# Discarding extra columns and moving 'Page' to front:
merged_df = merged_df[['Page'] + list(list_cols)]

Since GoLive Date has been moved, we need to turn it into a date, again, but it is already formated correctly from our ealier work. 

We are then ready to use the column dates and `Go Live Dates` to discard PPVs before a center was live.

In [ ]:
merged_df['GoLive Date'] = pd.to_datetime(merged_df['GoLive Date'], errors='coerce')

# GoLive Date Checks:
for index, row in merged_df.iterrows():
    go_live_date = row['GoLive Date']
    if pd.isnull(go_live_date):
        # If GoLive Date is missing, set all date columns to 0
        merged_df.loc[index, list_cols] = ''
    else:
        for col in list_cols:
            col_date = pd.to_datetime(col, errors='coerce') + timedelta(1)
            if go_live_date > col_date:
                merged_df.at[index, col] = ''

Because of the quirks of Google Sheets, I'm going to add empty columns to the date of the dataframe to ensure the total number of columns is always equal to 33. This ensures that Dave's formulas for finding the PPVs still work.

In [ ]:
def ensure_32_columns(df):
    current_column_count = df.shape[1]
    columns_to_add = 33 - current_column_count

    for i in range(columns_to_add):
        df[f'Empty Column {current_column_count + i + 1}'] = pd.NA

    return df


merged_df = ensure_32_columns(merged_df)
merged_df = merged_df.where(pd.notnull(merged_df), '')

After dropping `Go Live Date`, or data is finalized and ready to be sent to the APRB sheet:

In [ ]:
finished_data = merged_df.drop(columns=['GoLive Date'])
finished_data

,Page,2024-01-01,2024-01-02,2024-01-03,2024-01-04,2024-01-05,2024-01-06,2024-01-07,2024-01-08,Empty Column 11,...,Empty Column 24,Empty Column 25,Empty Column 26,Empty Column 27,Empty Column 28,Empty Column 29,Empty Column 30,Empty Column 31,Empty Column 32,Empty Column 33
0,/1000-islands/,28.0,18.0,18.0,12.0,10.0,13.0,27.0,6.0,,...,,,,,,,,,,
1,/90210-recovery-california/,5.0,2.0,4.0,4.0,2.0,6.0,7.0,,,...,,,,,,,,,,
2,/abhasa-rehabilitation-india/,15.0,11.0,7.0,8.0,7.0,12.0,16.0,1.0,,...,,,,,,,,,,
3,/abhasa-rehabilitation-thondamuthur-india/,8.0,6.0,3.0,6.0,1.0,12.0,11.0,,,...,,,,,,,,,,
4,/abhasa-wellness-retreat/,7.0,5.0,7.0,2.0,3.0,13.0,12.0,1.0,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,/wilmington-treatment-center-north-carolina/,1.0,3.0,,2.0,2.0,3.0,1.0,,,...,,,,,,,,,,
251,/wish-recovery-california/,21.0,13.0,16.0,17.0,17.0,33.0,30.0,,,...,,,,,,,,,,
252,/within-center-austin-texas/,4.0,3.0,11.0,13.0,8.0,8.0,19.0,1.0,,...,,,,,,,,,,
253,/zeus-rehab-warsaw-poland/,,,,3.0,7.0,13.0,12.0,,,...,,,,,,,,,,


### Send Data to ADPB Sheet

Sending data to AdvertisingPartners_RatesAndBudgets using the GoogleSheets API and pygsheets:

In [ ]:
write_to_gsheet('/work/keveldata-5f76a71e9aaf.json','1G5Mo3wrT_SrbFt5lyjKDne4C-Hj0cfjxCaW9mGPLQXU',"PPV Data",finished_data)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6d4ba941-f702-478c-87bf-a3e3e8035885' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>